In [1]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder, Imputer, StandardScaler

In [2]:
spark = SparkSession.builder.appName('Stroke').getOrCreate()

In [3]:
df = spark.read.csv('healthcare-dataset-stroke-data.csv', header = True, inferSchema=True)

In [4]:
df.show(5)

+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
|   id|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level| bmi| smoking_status|stroke|
+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+------+
| 9046|  Male|67.0|           0|            1|         Yes|      Private|         Urban|           228.69|36.6|formerly smoked|     1|
|51676|Female|61.0|           0|            0|         Yes|Self-employed|         Rural|           202.21| N/A|   never smoked|     1|
|31112|  Male|80.0|           0|            1|         Yes|      Private|         Rural|           105.92|32.5|   never smoked|     1|
|60182|Female|49.0|           0|            0|         Yes|      Private|         Urban|           171.23|34.4|         smokes|     1|
| 1665|Female|79.0|           1|            0|         

In [5]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- ever_married: string (nullable = true)
 |-- work_type: string (nullable = true)
 |-- Residence_type: string (nullable = true)
 |-- avg_glucose_level: double (nullable = true)
 |-- bmi: string (nullable = true)
 |-- smoking_status: string (nullable = true)
 |-- stroke: integer (nullable = true)



In [6]:
df_prediction = df.select('gender','age','hypertension','heart_disease','Residence_type','avg_glucose_level','bmi','smoking_status','stroke')

In [7]:
df_prediction.show(5)

+------+----+------------+-------------+--------------+-----------------+----+---------------+------+
|gender| age|hypertension|heart_disease|Residence_type|avg_glucose_level| bmi| smoking_status|stroke|
+------+----+------------+-------------+--------------+-----------------+----+---------------+------+
|  Male|67.0|           0|            1|         Urban|           228.69|36.6|formerly smoked|     1|
|Female|61.0|           0|            0|         Rural|           202.21| N/A|   never smoked|     1|
|  Male|80.0|           0|            1|         Rural|           105.92|32.5|   never smoked|     1|
|Female|49.0|           0|            0|         Urban|           171.23|34.4|         smokes|     1|
|Female|79.0|           1|            0|         Rural|           174.12|  24|   never smoked|     1|
+------+----+------------+-------------+--------------+-----------------+----+---------------+------+
only showing top 5 rows



In [8]:
df_prediction.dtypes

[('gender', 'string'),
 ('age', 'double'),
 ('hypertension', 'int'),
 ('heart_disease', 'int'),
 ('Residence_type', 'string'),
 ('avg_glucose_level', 'double'),
 ('bmi', 'string'),
 ('smoking_status', 'string'),
 ('stroke', 'int')]

In [9]:
df_prediction.groupBy('smoking_status').count().show()

+---------------+-----+
| smoking_status|count|
+---------------+-----+
|         smokes|  789|
|        Unknown| 1544|
|   never smoked| 1892|
|formerly smoked|  885|
+---------------+-----+



In [10]:
df_prediction.groupBy(['gender','Residence_type','smoking_status']).count().show()

+------+--------------+---------------+-----+
|gender|Residence_type| smoking_status|count|
+------+--------------+---------------+-----+
|Female|         Urban|         smokes|  243|
|Female|         Rural|formerly smoked|  227|
|  Male|         Urban|         smokes|  183|
|Female|         Urban|   never smoked|  618|
|Female|         Rural|   never smoked|  611|
|Female|         Urban|        Unknown|  418|
|  Male|         Rural|formerly smoked|  200|
|Female|         Rural|         smokes|  209|
| Other|         Rural|formerly smoked|    1|
|  Male|         Urban|        Unknown|  364|
|  Male|         Rural|   never smoked|  350|
|Female|         Urban|formerly smoked|  250|
|  Male|         Urban|   never smoked|  313|
|  Male|         Rural|         smokes|  154|
|Female|         Rural|        Unknown|  418|
|  Male|         Rural|        Unknown|  344|
|  Male|         Urban|formerly smoked|  207|
+------+--------------+---------------+-----+



In [11]:
df_prediction.select('smoking_status').distinct().show()

+---------------+
| smoking_status|
+---------------+
|         smokes|
|        Unknown|
|   never smoked|
|formerly smoked|
+---------------+



In [12]:
genderEncoder = StringIndexer(inputCols=['gender','Residence_type','smoking_status'], outputCols=['indexer_gender','indexer_Residence_type','indexer_smoking_status']).fit(df_prediction)

In [13]:
df_prediction_indexer = genderEncoder.transform(df_prediction)

In [14]:
df_prediction_indexer.show(5)

+------+----+------------+-------------+--------------+-----------------+----+---------------+------+--------------+----------------------+----------------------+
|gender| age|hypertension|heart_disease|Residence_type|avg_glucose_level| bmi| smoking_status|stroke|indexer_gender|indexer_Residence_type|indexer_smoking_status|
+------+----+------------+-------------+--------------+-----------------+----+---------------+------+--------------+----------------------+----------------------+
|  Male|67.0|           0|            1|         Urban|           228.69|36.6|formerly smoked|     1|           1.0|                   0.0|                   2.0|
|Female|61.0|           0|            0|         Rural|           202.21| N/A|   never smoked|     1|           0.0|                   1.0|                   0.0|
|  Male|80.0|           0|            1|         Rural|           105.92|32.5|   never smoked|     1|           1.0|                   1.0|                   0.0|
|Female|49.0|         

In [15]:
df = df_prediction_indexer.select('indexer_gender','age','hypertension','heart_disease','indexer_Residence_type','avg_glucose_level','bmi','indexer_smoking_status','stroke')

In [16]:
df.show(5)

+--------------+----+------------+-------------+----------------------+-----------------+----+----------------------+------+
|indexer_gender| age|hypertension|heart_disease|indexer_Residence_type|avg_glucose_level| bmi|indexer_smoking_status|stroke|
+--------------+----+------------+-------------+----------------------+-----------------+----+----------------------+------+
|           1.0|67.0|           0|            1|                   0.0|           228.69|36.6|                   2.0|     1|
|           0.0|61.0|           0|            0|                   1.0|           202.21| N/A|                   0.0|     1|
|           1.0|80.0|           0|            1|                   1.0|           105.92|32.5|                   0.0|     1|
|           0.0|49.0|           0|            0|                   0.0|           171.23|34.4|                   3.0|     1|
|           0.0|79.0|           1|            0|                   1.0|           174.12|  24|                   0.0|     1|


In [17]:
df.dtypes

[('indexer_gender', 'double'),
 ('age', 'double'),
 ('hypertension', 'int'),
 ('heart_disease', 'int'),
 ('indexer_Residence_type', 'double'),
 ('avg_glucose_level', 'double'),
 ('bmi', 'string'),
 ('indexer_smoking_status', 'double'),
 ('stroke', 'int')]

In [18]:
from pyspark.sql.functions import col

df_na = df.filter(col("BMI") == "N/A")
df_na.groupBy('BMI').count().show()

+---+-----+
|BMI|count|
+---+-----+
|N/A|  201|
+---+-----+



In [19]:
df_filtered = df.where(col("BMI") != "N/A")
df_filtered.show(10)

+--------------+----+------------+-------------+----------------------+-----------------+----+----------------------+------+
|indexer_gender| age|hypertension|heart_disease|indexer_Residence_type|avg_glucose_level| bmi|indexer_smoking_status|stroke|
+--------------+----+------------+-------------+----------------------+-----------------+----+----------------------+------+
|           1.0|67.0|           0|            1|                   0.0|           228.69|36.6|                   2.0|     1|
|           1.0|80.0|           0|            1|                   1.0|           105.92|32.5|                   0.0|     1|
|           0.0|49.0|           0|            0|                   0.0|           171.23|34.4|                   3.0|     1|
|           0.0|79.0|           1|            0|                   1.0|           174.12|  24|                   0.0|     1|
|           1.0|81.0|           0|            0|                   0.0|           186.21|  29|                   2.0|     1|


In [20]:
df.printSchema()

root
 |-- indexer_gender: double (nullable = false)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- indexer_Residence_type: double (nullable = false)
 |-- avg_glucose_level: double (nullable = true)
 |-- bmi: string (nullable = true)
 |-- indexer_smoking_status: double (nullable = false)
 |-- stroke: integer (nullable = true)



In [21]:
df_pandas = df.toPandas().replace('N/A',0).astype(float)

In [22]:
type(df)

pyspark.sql.dataframe.DataFrame

In [23]:
type(df_pandas)

pandas.core.frame.DataFrame

In [28]:
df_pandas.head(5)

,indexer_gender,age,hypertension,heart_disease,indexer_Residence_type,avg_glucose_level,bmi,indexer_smoking_status,stroke
0,1.0,67.0,0.0,1.0,0.0,228.69,36.6,2.0,1.0
1,0.0,61.0,0.0,0.0,1.0,202.21,0.0,0.0,1.0
2,1.0,80.0,0.0,1.0,1.0,105.92,32.5,0.0,1.0
3,0.0,49.0,0.0,0.0,0.0,171.23,34.4,3.0,1.0
4,0.0,79.0,1.0,0.0,1.0,174.12,24.0,0.0,1.0


In [29]:
df_new_pyspark = spark.createDataFrame(df_pandas)
type(df_new_pyspark)

pyspark.sql.dataframe.DataFrame

In [30]:
df_new_pyspark.printSchema()

root
 |-- indexer_gender: double (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: double (nullable = true)
 |-- heart_disease: double (nullable = true)
 |-- indexer_Residence_type: double (nullable = true)
 |-- avg_glucose_level: double (nullable = true)
 |-- bmi: double (nullable = true)
 |-- indexer_smoking_status: double (nullable = true)
 |-- stroke: double (nullable = true)



In [32]:
df_new_pyspark.show()

Py4JJavaError: An error occurred while calling o208.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 26.0 failed 1 times, most recent failure: Lost task 0.0 in stage 26.0 (TID 21) (DESKTOP-K5H262C executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheck